In [1]:
import numba
import numpy as np


**Funciones de prueba:**

In [2]:
@numba.njit
def Sphere(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función de la esfera en el vector.
    Se usa en el intervalo de búsqueda [-5.12, 5.12] y su óptimo global está en f([0,..,0])=0

    Ejemplo de uso:
        > Sphere(np.array([0, 0]))

        > 0
    """
    suma = 0.0  # Usar un float para la suma
    for x in arr:
        suma += x ** 2
    return suma

intervaloSphere = (-5.12, 5.12)


# Ejemplo de uso (y compilación AOT)
resultado = Sphere(np.array([1.0, 2.0, 3.0]))
print(resultado)


14.0


In [3]:
@numba.njit
def Ackley(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función Ackley en el vector.
    Se usa en el intervalo de búsqueda [-30, 30] y su óptimo global está en f([0,..,0])=0.

    Ejemplo de uso:
        > Ackley(np.array([0, 0]))

        > 0
    """
    suma1 = 0.0
    suma2 = 0.0
    n = arr.shape[0]  # Usar el tamaño del array de NumPy
    for x in arr:
        suma1 += x ** 2
        suma2 += np.cos(2 * np.pi * x)
    
    # Calcular el resultado de la función Ackley
    term1 = 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt(suma1 / n))
    term2 = np.exp(suma2 / n)
    return term1 - term2

intervaloAckley = (-30, 30)

# Ejemplo de uso (y compilación AOT)
resultado = Ackley(np.array([0, 0]))
print(resultado)  # Debería imprimir un valor cercano a 0


-4.440892098500626e-16


In [4]:
@numba.njit
def Griewank(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función Griewank en el vector.
    Se usa en el intervalo de búsqueda [-600, 600] y su óptimo global está en f([0,..,0])=0.

    Ejemplo de uso:
        > Griewank(np.array([0, 0]))

        > 0
    """
    suma = 0.0
    m = 1.0  # Inicializar en 1 para la multiplicación de cosenos
    n = arr.shape[0]  # Obtener el número de elementos en el array

    for i in range(1, n + 1):
        x_i = arr[i - 1]  # Acceder al elemento en el índice correspondiente
        suma += (x_i ** 2) / 4000.0  # Sumar la parte de la función
        m *= np.cos(x_i / np.sqrt(i))  # Multiplicar por el coseno

    return 1 + suma - m

intervaloGriewank = (-600, 600)

# Ejemplo de uso (y compilación AOT)
resultado = Griewank(np.array([0, 0]))
print(resultado)  # Debería imprimir 0.0


0.0


In [5]:
@numba.njit
def Rastrigin(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
    Tiene mínimo global en f([0,..,0]).

    Ejemplo de uso:
        > Rastrigin(np.array([2, 3, 6, 7]))

        > 98.0
    """
    n = arr.shape[0]  # Obtener el número de elementos en el array
    suma = 0.0

    for x in arr:
        suma += x ** 2 - 10 * np.cos(2 * np.pi * x)  # Calcular la suma de la función Rastrigin

    return 10 * n + suma  # Retornar el resultado final

intervaloRastrigin = (-5.12, 5.12)

# Ejemplo de uso (y compilación AOT)
resultado = Rastrigin(np.array([2, 3, 6, 7]))
print(resultado)  # Debería imprimir 98.0


98.0


In [6]:
@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

# Ejemplo de uso (y compilación AOT)
resultado = Rosenbrock(np.array([1, 1]))
print(resultado)  # Debería imprimir 0.0


0.0


**Funciones del algoritmo genético.**

In [7]:
kindividuos = 35 
dimension = 10
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 20
nCortes = 15
probaMutacion = 0.5
iteraciones = 1000


In [8]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

@numba.njit
def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valores aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   return poblacionMatriz # No hacemos la conversión en lista de listas
                          # porque necesitamos de `numba`.


In [9]:
# Ejemplo de uso (y compilación AOT)

pob = Generar_Poblacion(kindividuos, dimension, intervalo)
print(pob[0])


[ 1.21280886  0.55683995 -1.31769236 -1.44958367 -0.69175333 -0.76202639
  1.89601948  0.74246922  0.08820668 -1.78711215]


In [10]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y retorna una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''


# `aptitud` es un parámetro opcional sólo para elegir qué retorna la 
# función.
@numba.njit
def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion, aptitud=1):
    
    # Necesitamos el número de individuos de la población:
    numIndividuos = len(poblacion) 
    
    # Generamos un array con las evaluaciones en la función objetivo
    evaluaciones = np.empty(numIndividuos) # Primero generamos un array
                                           # vacío de tamaño adecuado.
    for i in range(numIndividuos):
        evaluaciones[i] = funcion(poblacion[i])

    # Generamos un array de las aptitudes de los individuos en la 
    # población.
    aptitudes = np.exp(1e-9 * (-evaluaciones))

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = np.sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = aptitudes / sumaAptitudes

    # Inicializamos mejorApti y promedioApti
    mejorApti = np.nan
    promedioApti = np.nan

    # Si aptitud es distinto de 0, calculamos mejor aptitud y promedio de aptitudes
    if aptitud != 0:
        mejorApti = aptitudes[mejorIndice]
        promedioApti = np.mean(aptitudes)

    return probabilidades, masApto, mejorApti, promedioApti


In [11]:
# Ejemplo de uso (y compilación AOT)
probas, masApto, mejorApti, promedioApti = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, pob)

print(probas[0], masApto[0], mejorApti, promedioApti)


0.014285724519501062 -0.6473520432798086 0.9999996534774201 0.9999956366031528


In [12]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos retorna la que corresponde a la de las 
   probabilidades acumulativas.'''

@numba.njit
def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = np.cumsum(probabilidades)

    return probasAcumulativas # Retornamos las `probasAcumulativas` como
                              # un arreglo de NumPy (igual es iterable).


In [13]:
# Ejemplo de uso (y compilación AOT)
qis = Probas_Acumulativas(probas)
print(qis[16])


0.24285716834346574


Padres mediante el uso de la ruleta:

In [14]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

@numba.njit
def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = np.random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   
   # Recorremos las probabilidades acumuladas para seleccionar el padre
   for i in range(len(probasAcumuladas)):
      if r <= probasAcumuladas[i] or np.isclose(r, probasAcumuladas[i], atol=tol):
         return i  # Índice del individuo seleccionado
               
   return -1  # En caso de que no se seleccione, aunque esto no debería suceder.


In [15]:
'''La segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k nuevos individuos, así que se van a 
   elegir k parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades acumulativas y un máximo de intentos para intentar evitar
   la generación de clones (en este caso, vamos a considerar un máximo
   de 100 para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).
   Retorna la lista de parejas de padres elegidos.'''

@numba.njit
def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    
    numIndividuos = len(individuos)

    # Validación: si no hay suficientes individuos
    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return np.empty((0, 2, individuos.shape[1]))
        # Regresamos un array vacío

    # Como len(individuos) = 2k, le pedimos que elija len(individuos)/2 
    # parejas de padres.
    parejasNecesarias = numIndividuos // 2
    
    
    parejas_idxs = np.empty((parejasNecesarias, 2), dtype=np.int32)
    # Creamos un array para los índices de tamaño adecuado.
    # Como queremos parejas, el array tiene dos columnas, la idea es que
    # cada fila del array represente una pareja de padres.

    '''En otras palabras, `parejas_idxs` tendrá la forma:
       [[padre1_idx1, padre2_idx1],
       [padre1_idx2, padre2_idx2],
       [padre1_idx3, padre2_idx3],
       ...
       [padre1_idxN, padre2_idxN]]'''


    for i in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)
            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salimos del ciclo
            intentos += 1 # No encontramos un padre distinto, hacemos 
                          # otro intento.
            if intentos >= maxIntentos: # Rebasamos los intentos que
                                        # teníamos permitidos.
                padre2_idx = padre1_idx # Aceptamos el clon.
                break

        parejas_idxs[i, 0] = padre1_idx # Primer padre se asigna en la
                                        # fila `i` de la primer columna.
        parejas_idxs[i, 1] = padre2_idx # Segundo padre se asigna en la
                                        # fila `i` de la segunda columna.

    # Ahora construimos el array de parejas usando los índices generados
    # para almacenar las parejas de individuos.
    parejas = np.empty((parejasNecesarias, 2, individuos.shape[1]), dtype=individuos.dtype)
    
    for i in range(parejasNecesarias):
        parejas[i, 0] = individuos[parejas_idxs[i, 0]]  # Primer padre
        parejas[i, 1] = individuos[parejas_idxs[i, 1]]  # Segundo padre

    
    return parejas


In [16]:
# Ejemplo de uso (y compilación AOT)
padres = Generar_Parejas(pob, qis)
print(padres[0])


[[-1.37304197  1.91727272 -0.3704863   0.98484994  2.03615133  0.29972759
   0.57770423 -0.88412343  0.9699244   2.02487877]
 [-1.55254123  1.07272438 -0.07652456 -1.28134201  0.15261907 -1.35750601
   1.60585771  0.8768048   1.60202561 -1.77170767]]


Funciones auxiliares para la codificación en binario:

In [17]:
@numba.njit
def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando n_bit bits
    y una partición uniforme en [a, b].

    Parámetros:
    x: número real a codificar.
    n_bit: número de bits a utilizar.
    intervalo: (a, b), extremos del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con n_bit bits
    en el intervalo [a, b].
    """
    a, b = intervalo

    # Calcula la precisión de la representación.
    precision = (b - a) / ((2 ** n_bit) - 1)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = np.zeros(n_bit, dtype=np.int32)
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    return x_binario

@numba.njit
def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios
    utilizando n_bit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    n_bit: Número de bits a utilizar para las entradas de nuestro arreglo.
    intervalo: (a, b), extremos del intervalo.

    Retorno:
    Arreglo de arreglos binarios, donde cada sub-arreglo representa un número real
    codificado en binario.
    """
    dim_x = vector_reales.shape[0]

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = np.empty((dim_x, n_bit), dtype=np.int32)

    for i in range(dim_x):
        vector_binario[i] = codifica_real(vector_reales[i], n_bit, intervalo)

    return vector_binario


In [18]:
@numba.njit
def Padres_Binarios(parejas, nBits, intervalo):
    """
    Codifica parejas de vectores en binario.
    """
    numParejas = parejas.shape[0]
    parejasBinarias = np.empty((numParejas, 2, parejas.shape[2], nBits), dtype=np.int32)

    for i in range(numParejas):
        padre1 = parejas[i, 0]  # Vector 1
        padre2 = parejas[i, 1]  # Vector 2

        # Codifica cada padre en binario y lo almacena.
        parejasBinarias[i, 0] = codifica_vector(padre1, nBits, intervalo)
        parejasBinarias[i, 1] = codifica_vector(padre2, nBits, intervalo)

    return parejasBinarias


In [19]:
# Ejemplo de uso (y compilación AOT)
padresBinarios = Padres_Binarios(padres, nBits, intervalo)
print(padresBinarios[0,0])


[[0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 0 1 0 1]
 [1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 0 0]
 [0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1]
 [1 0 1 1 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1]
 [1 0 0 1 0 0 1 0 1 0 1 1 1 0 1 1 1 0 0 1]
 [1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1]
 [0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0]
 [1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 1]
 [1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1]]


In [20]:
'''Función que realiza la cruza de n puntos. Recibe como parámetros: el
   array de los padres (en arreglos binarios), el número de cortes que 
   se quieren hacer por cromosoma.Retorna una lista de 2k hijos (como 
   arreglos binarios).'''

@numba.njit
def Cruza_N_Puntos_np_numba(padres, nPuntos):
    numParejas = padres.shape[0]  # Número de parejas de padres
    numCromosomas = padres[0, 0].shape[0]  # Número de cromosomas por padre (filas)
    numGenes = padres[0, 0].shape[1]  # Número de genes por cromosoma (columnas)

    # Inicializamos un array vacío para almacenar los hijos
    hijos = np.empty((2 * numParejas, numCromosomas, numGenes), dtype=np.int32)

    # Iteramos sobre cada pareja de padres
    for pareja_idx in range(numParejas):
        padre1 = padres[pareja_idx, 0]
        padre2 = padres[pareja_idx, 1]

        # Generamos los arrays vacíos para los hijos de la pareja actual
        hijo1 = np.empty_like(padre1)
        hijo2 = np.empty_like(padre2)

        # Generar n puntos de corte únicos
        puntosCorte = np.random.choice(np.arange(1, numGenes), nPuntos, replace=False)
                      # `replace=False` es para garantizar que los puntos de corte sí
                      # sean diferentes.
        puntosCorte.sort()  # Asegura que los puntos de corte están ordenados.

        # Iteramos sobre cada cromosoma de los padres
        for cromo_idx in range(numCromosomas):
            cromo1 = padre1[cromo_idx]
            cromo2 = padre2[cromo_idx]

            # Alternamos segmentos entre los padres
            ultimo_punto = 0
            switch = False

            for punto in np.append(puntosCorte, numGenes):  # Añadimos el último punto
                if switch:
                    hijo1[cromo_idx, ultimo_punto:punto] = cromo2[ultimo_punto:punto]
                    hijo2[cromo_idx, ultimo_punto:punto] = cromo1[ultimo_punto:punto]
                else:
                    hijo1[cromo_idx, ultimo_punto:punto] = cromo1[ultimo_punto:punto]
                    hijo2[cromo_idx, ultimo_punto:punto] = cromo2[ultimo_punto:punto]

                switch = not switch
                ultimo_punto = punto

        # Guardamos los hijos generados en el array de hijos
        hijos[2 * pareja_idx] = hijo1
        hijos[2 * pareja_idx + 1] = hijo2

    return hijos


In [21]:
# Ejemplo de uso (y compilación AOT)
hijosBinarios = Cruza_N_Puntos_np_numba(padresBinarios, nCortes)
print(hijosBinarios[0])


[[0 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1]
 [1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0]
 [0 1 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1]
 [1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0]
 [1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1]
 [1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1 0 0 0]
 [1 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 0]
 [0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 1]
 [1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1]
 [1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0]]


In [22]:
@numba.njit
def Mutador_1_flip_np_numba(hijosBinarios, probaMutacion):
    
    # Obtenemos el número de individuos y cromosomas
    numHijos = hijosBinarios.shape[0]
    numCromosomas = hijosBinarios.shape[1]
    
    numBits = hijosBinarios.shape[2]

    hijosMutados = np.copy(hijosBinarios)
    
    # Recorremos cada individuo
    for i in range(numHijos):
        # Recorremos cada cromosoma del individuo
        for j in range(numCromosomas):
            # Generamos un número aleatorio
            if np.random.rand() < probaMutacion:
                # Elegimos un bit al azar dentro del cromosoma
                bit_a_flip = np.random.randint(0, numBits)
                # Mutar el bit
                hijosMutados[i, j, bit_a_flip] ^= 1  # Flip bit
                # Modificamos directamente sobre hijosBinarios porque
                # a los que realmente consideraremos como hijos en cada
                # generación es a los generados después de la reproducción
                # y mutación.

    return hijosMutados


In [23]:
# Ejemplo de uso (y compilación AOT)
hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)
print(hijosMutados[0])


[[0 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1]
 [1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0]
 [0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1]
 [1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0]
 [1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 1]
 [1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1 0 0 0]
 [1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0]
 [0 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1]
 [1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1]
 [1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 0 1 1 1 0]]


In [24]:
@numba.njit
def Hijos_Decodificados(hijosMutados, nBits, intervalo):
    """
    Decodifica un array de hijos mutados en números reales.

    Parámetros:
    hijosMutados: np.array de hijos mutados, donde cada hijo es un vector de bits
    nBits: número de bits a utilizar para cada número
    intervalo: de la forma [a, b]

    Retorno:
    np.array de números reales decodificados
    """
    
    num_hijos = hijosMutados.shape[0]      # Obtenemos el número de hijos
    num_cromosomas = hijosMutados.shape[1]  # Obtenemos el número de cromosomas
    
    # Crea un array vacío para los hijos decodificados
    hijosDecodificados = np.empty((num_hijos, num_cromosomas))  # De la forma correcta

    for i in range(num_hijos):
        for j in range(num_cromosomas):
            hijoMutado = hijosMutados[i, j]  # Acceder al cromosoma
            
            # Decodificación directa del cromosoma
            a, b = intervalo
            precision = (b - a) / ((2 ** nBits) - 1)  # Calcula la precisión de la representación
            # Convierte el vector de bits en un número entero
            indice = 0
            for bit in range(nBits):
                indice += hijoMutado[bit] * (2 ** (nBits - 1 - bit))  # Cálculo del valor decimal
            hijoDecodificado = a + indice * precision  # Calcula el número real decodificado
            
            # Almacena el resultado en el array de salida
            hijosDecodificados[i, j] = hijoDecodificado

    return hijosDecodificados


In [25]:
# Ejemplo de uso (y compilación AOT)
hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
print(hijosDecodificados[0])


[-1.11254208  1.59280035 -0.61495566  0.7328464   0.68867839  0.05822076
  1.60560505 -1.5850269   1.73639033  0.95630755]


Funciones que realizan el reemplazo generacional:

In [26]:
'''Función que realiza el reemplazo con elitismo.'''

@numba.njit
def Reemplazo_Elitismo(hijosDecodificados, masApto):

    numHijos, dimension = hijosDecodificados.shape

    # Elegimos el índice de un hijo al azar para eliminarlo:
    idx_aleatorio = np.random.randint(0, numHijos)

    # Creamos un nuevo array sin el hijo borrado:
    hijosReducidos = np.empty((numHijos-1, dimension))
    contador = 0
    for i in range(numHijos):
        if i != idx_aleatorio:
            hijosReducidos[contador, :] = hijosDecodificados[i, :]
            contador += 1

    # Creamos un nuevo array con el más apto antes de la reproducción:
    siguienteGeneracion = np.empty((numHijos, dimension))
    for i in range(numHijos-1):
        siguienteGeneracion[i, :] = hijosReducidos[i, :]
    siguienteGeneracion[numHijos-1, :] = masApto

    return siguienteGeneracion


In [27]:
# Ejemplo de uso (y compilación AOT)
siguienteGen = Reemplazo_Elitismo(hijosDecodificados, masApto)
print(siguienteGen[-1])


[-0.64735204 -0.2230656   0.45692464  0.35117501  0.19631298  0.83231641
  0.69957579 -0.004212    0.0200008  -1.38189235]


In [28]:
'''Función que realiza el reemplazo generacional completo.'''

@numba.njit
def Reemplazo_Generacional(hijosDecodificados):
    
    # Hacemos el reemplazo generacional completo:
    siguienteGen = np.copy(hijosDecodificados)

    return siguienteGen
    

In [29]:
# Ejemplo de uso (y compilación AOT)
siguienteGen = Reemplazo_Generacional(hijosDecodificados)
print(siguienteGen[-1])


[ 0.42012345  1.02672949  1.03776466 -0.30518974  1.84919903 -0.79953006
  0.87317466  0.58339313 -0.34864291 -1.61865975]


In [30]:
'''Función que realiza el reemplazo de los peores.'''

@numba.njit
def Reemplazo_Peores(hijosDecodificados, poblacion, funcion):

    # Hacemos un array donde estén concatenados la población
    # actual y los hijos:
    poblacionTotal = np.vstack((poblacion, hijosDecodificados))

    # Calculamos las evaluaciones de todos los individuos en 
    # la `poblacionTotal`
    evaluaciones = np.empty(len(poblacionTotal)) # Array vacío
    for i in range(len(poblacionTotal)):
        evaluaciones[i] = funcion(poblacionTotal[i])

    # Calculamos las aptitudes de los individuos:
    aptitudes = np.exp(1e-9 * (-evaluaciones))

    # Ordenamos a los individuos de acuerdo a su aptitud en 
    # orden descendente:
    indicesOrdenados = np.argsort(aptitudes)[::-1] 
    # El ordenamiento es por índices.

    # Seleccionamos a los mejores individuos (manteniendo
    # el tamaño original de la población).
    nuevaPoblacion = poblacionTotal[indicesOrdenados[:len(poblacion)]]

    return nuevaPoblacion


In [33]:
# Ejemplo de uso (y compilación AOT)
siguienteGen = Reemplazo_Peores(hijosDecodificados, pob, funcion)
print(siguienteGen[-1])


[ 1.46381585  0.53408059 -0.23820921 -0.82923321  1.83148026  1.7441169
  0.70187762 -1.90269908  1.90051548 -1.18353824]


Algoritmos genéticos:

In [31]:
@numba.njit
def AG_Reemplazo_Elitismo(funcion, intervalo, dimension, 
                          kindividuos, nBits, nCortes, 
                          probaMutacion, iteraciones):

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo)

    # Repetiremos esto mientras tengamos iteraciones:
    for i in range(iteraciones):

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto, mejorApti, promedioApti = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres:
        padres = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruza_N_Puntos_np_numba(padresBinarios, nCortes)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
        
        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Reemplazo_Elitismo(hijosDecodificados, masApto)

        # Nos interesa la evaluación del más apto:
        masAptoEval = funcion(masApto)

    return masAptoEval, mejorApti, promedioApti


In [32]:
masAptoEval, mejorApti, promedioApti = AG_Reemplazo_Elitismo(funcion, intervalo, dimension, 
                                                         kindividuos, nBits, nCortes, 
                                                         probaMutacion, iteraciones)

print(masAptoEval)
print(mejorApti)
print(promedioApti)


22.02114629433964
0.999999977978854
0.9999958307780004


In [33]:
@numba.njit
def AG_Reemplazo_Generacional(funcion, intervalo, dimension, 
                              kindividuos, nBits, nCortes, 
                              probaMutacion, iteraciones):

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo)

    # Repetiremos esto mientras tengamos iteraciones:
    for i in range(iteraciones):

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto, mejorApti, promedioApti = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres:
        padres = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruza_N_Puntos_np_numba(padresBinarios, nCortes)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
        
        # Generamos la siguiente generación, con reemplazo generacional completo.
        poblacion = Reemplazo_Generacional(hijosDecodificados)

        # Nos interesa la evaluación del más apto:
        masAptoEval = funcion(masApto)

    return masAptoEval, mejorApti, promedioApti


In [34]:
masAptoEval, mejorApti, promedioApti = AG_Reemplazo_Generacional(funcion, intervalo, dimension, 
                                                                 kindividuos, nBits, nCortes, 
                                                                 probaMutacion, iteraciones)

print(masAptoEval)
print(mejorApti)
print(promedioApti)


459.00259973959385
0.9999995409975057
0.9999963321530329


In [35]:
@numba.njit
def AG_Reemplazo_Peores(funcion, intervalo, dimension, 
                        kindividuos, nBits, nCortes, 
                        probaMutacion, iteraciones):

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo)

    # Repetiremos esto mientras tengamos iteraciones:
    for i in range(iteraciones):

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto, mejorApti, promedioApti = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres:
        padres = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruza_N_Puntos_np_numba(padresBinarios, nCortes)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip_np_numba(hijosBinarios, probaMutacion)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)
        
        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Reemplazo_Peores(hijosDecodificados, poblacion, funcion)

        # Nos interesa la evaluación del más apto:
        masAptoEval = funcion(masApto)

    return masAptoEval, mejorApti, promedioApti


In [36]:
masAptoEval, mejorApti, promedioApti = AG_Reemplazo_Peores(funcion, intervalo, dimension, 
                                                           kindividuos, nBits, nCortes, 
                                                           probaMutacion, iteraciones)

print(masAptoEval)
print(mejorApti)
print(promedioApti)


7.619655458305994
0.9999999923803445
0.9999999923801951


In [37]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")


funcion = Rosenbrock

intervalo = intervaloRosenbrock

dimension = 10
kindividuos = 35 # Tendremos una población del doble
nBits = 20
nCortes = 16
probaMutar = 0.01
iteraciones = 1000

ejecuciones = 30
j = 0



while j <= ejecuciones:
    masAptoEval, mejorApti, promedioApti = AG_Reemplazo_Elitismo(funcion, intervalo, dimension,   
                                                                 kindividuos, nBits, nCortes,     
                                                                 probaMutacion, iteraciones)
    #print(f"Rosenbrock:, {str(masApto):<10}, {funcion(masApto)}")
    j += 1


Semilla generada: 1728871815
Hora de generación: 2024-10-13 20:10:15



In [41]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")


funcion = Rosenbrock

intervalo = intervaloRosenbrock

dimension = 10
kindividuos = 35 # Tendremos una población del doble
nBits = 20
nCortes = 16
probaMutar = 0.01
iteraciones = 1000

ejecuciones = 30
j = 0



while j <= ejecuciones:
    masAptoEval, mejorApti, promedioApti = AG_Reemplazo_Generacional(funcion, intervalo, dimension, 
                                                                     kindividuos, nBits, nCortes, 
                                                                     probaMutacion, iteraciones)
    #print(f"Rosenbrock:, {str(masApto):<10}, {funcion(masApto)}")
    j += 1


Semilla generada: 1728871883
Hora de generación: 2024-10-13 20:11:23



In [40]:
# Establecer una semilla fija
import time
from datetime import datetime

# Generar una semilla basada en la hora del sistema
seed = int(time.time()) % (2**32)
np.random.seed(seed)

# Imprimir la semilla y la hora actual
hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"Semilla generada: {seed}")
print(f"Hora de generación: {hora_actual}\n")


funcion = Rosenbrock

intervalo = intervaloRosenbrock

dimension = 10
kindividuos = 35 # Tendremos una población del doble
nBits = 20
nCortes = 16
probaMutar = 0.01
iteraciones = 1000

ejecuciones = 30
j = 0



while j <= ejecuciones:
    masAptoEval, mejorApti, promedioApti = AG_Reemplazo_Peores(funcion, intervalo, dimension, 
                                                               kindividuos, nBits, nCortes, 
                                                               probaMutacion, iteraciones)
    #print(f"Rosenbrock:, {str(masApto):<10}, {funcion(masApto)}")
    j += 1


Semilla generada: 1728871866
Hora de generación: 2024-10-13 20:11:06

